In [ ]:
#%% TRY TO SAVE DERIVATIVE FROM BIDS DATASET HERE:
from main_meg_qc import make_derivative_meg_qc
config_file_name = 'settings.ini'
raw = make_derivative_meg_qc(config_file_name)
raw

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mne
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch
import configparser
import ancpbids
import plotly
import mpld3

from main_meg_qc import make_derivative_meg_qc
from initial_meg_qc import get_all_config_params, sanity_check, initial_processing
from RMSE_meq_qc import RMSE_meg_qc
from PSD_meg_qc import PSD_meg_qc
from Peaks_manual_meg_qc import PP_manual_meg_qc
from Peaks_auto_meg_qc import PP_auto_meg_qc
from ECG_meg_qc import ECG_meg_qc
from EOG_meg_qc import EOG_meg_qc
from universal_html_report import keep_fig_derivs, make_joined_report
import time


In [ ]:
from main_meg_qc import get_all_config_params, initial_processing
all_qc_params=get_all_config_params('settings.ini')
#print(all_qc_params)

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003483/sub-009/ses-1/meg/sub-009_ses-1_task-deduction_run-1_meg.fif' #GOOD ECG CHANNEL
#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from lab/mikado/sub_HT05ND16/210811/mikado-1.fif' #NO ECG CHANNEL, GOOD RECONSTRUCT


#data_file='/Volumes/M2_DATA/MEG_QC_stuff/data/from lab/forrest_gump_meg/en04ns31_vp15/190524/vp15_block1-1.fif'  #BAD ECG CHANNEL, 2 good eog.

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds004229/sub-102/meg/sub-102_task-amnoise_meg.fif' #2EOG channels, both bad

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003703/sub-a68d5xp5/meg/sub-a68d5xp5_task-listeningToSpeech_run-01_meg.fif'
#2EOG channels, both bad

data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds004107/sub-mind002/ses-01/meg/sub-mind002_ses-01_task-auditory_meg.fif'
#normal psd


#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds004107/sub-mind002/ses-01/meg/sub-mind002_ses-01_task-auditory_meg.fif'
#EOG 061 bad (or rather unusual), EOG 062 good. Mne takes only the good channel and calculates events on base of it -  
# my average and other plots are only on base of 1 goodchannel automatically

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003682/sub-003/ses-01/meg/sub-003_ses-01_task-AversiveLearningReplay_run-01_meg.fif'

#dict_of_dfs_epoch, dict_epochs_mg, channels, raw_bandpass, raw_filtered_resampled, raw_cropped, raw, active_shielding_used = initial_processing(default_settings=all_qc_params['default'], filtering_settings=all_qc_params['Filtering'], epoching_params=all_qc_params['Epoching'], data_file=data_file)

raw = mne.io.read_raw_fif(data_file, allow_maxshield=True)
raw_cropped = raw.copy()
tmin_my_plot=200
tmax_my_plot=300
duration_my_plot=tmax_my_plot-tmin_my_plot
raw_cropped.crop(tmin=tmin_my_plot, tmax=tmax_my_plot)

#raw_cropped.drop_channels(ECG_channel_name)

raw_cropped

In [ ]:
freq_min=0.5
freq_max=100
n_fft = 2500
n_per_seg = 2500

psds, freqs = raw.compute_psd(method='welch', fmin=freq_min, fmax=freq_max, picks='mag', n_jobs=-1, n_fft=n_fft, n_per_seg=n_per_seg).get_data(return_freqs=True)

from PSD_meg_qc import Plot_periodogram

mag_ch_names = raw.copy().pick_types(meg='mag').ch_names if 'mag' in raw else None
grad_ch_names = raw.copy().pick_types(meg='grad').ch_names if 'grad' in raw else None
channels = {'mag': mag_ch_names, 'grad': grad_ch_names}

der=Plot_periodogram('mag', freqs, psds, channels['mag'])
der.content.show()

In [ ]:
#Average psd:

avg_psd=np.mean(psds,axis=0)

thresh=(max(avg_psd) - min(avg_psd)) / 20
pos_peak_locs, pos_peak_magnitudes = mne.preprocessing.peak_finder(avg_psd, extrema=1, thresh=thresh, verbose=False) 

pos_peak_locs, pos_peak_magnitudes=pos_peak_locs[1:], pos_peak_magnitudes[1:]
# remove the first one, because it is not actually a peak, but a start of psd curve.

fig = go.Figure()
fig.add_trace(go.Scatter(x=freqs, y=avg_psd, name='Average PSD'))
fig.add_trace(go.Scatter(x=freqs[pos_peak_locs], y=pos_peak_magnitudes, mode='markers', name='peaks'))
fig.show()

print(len(avg_psd))
freqs

print(pos_peak_magnitudes)


In [ ]:
# 2500: [5.08812970e-27 5.77794458e-27 5.19501531e-27 2.47311358e-27]
# 1000: [4.59409698e-27 3.47110933e-27 1.03527341e-27]

In [ ]:
from scipy.signal import chirp, find_peaks, peak_widths
import matplotlib.pyplot as plt

x=avg_psd.copy()
prominence=(max(avg_psd) - min(avg_psd)) / 10
peaks, _ = find_peaks(x, prominence=prominence)

results_full = peak_widths(x, peaks, rel_height=1)


plt.plot(x)
plt.plot(peaks, x[peaks], "x")

plt.hlines(*results_full[1:], color="C3")
plt.show()

In [ ]:

prominence=(max(avg_psd) - min(avg_psd)) / 10
peaks, _ = find_peaks(avg_psd, prominence=prominence)

widths, width_heights, left_ips, right_ips = peak_widths(avg_psd, peaks, rel_height=1)

#Plot signal, peaks and contour lines at which the widths where calculated
from PSD_meg_qc import Power_of_band

print('Central Freqs: ', freqs[peaks])
print('Central Amplitudes: ', avg_psd[peaks])
print('width_heights: ', width_heights)

ips_pair=[]
ips_l=[]
ips_r=[]
avg_psd_cut=avg_psd.copy()
avg_psd_only_peaks=avg_psd.copy()
avg_psd_only_peaks[:]=None
avg_psd_only_peaks_baselined=avg_psd.copy()
avg_psd_only_peaks_baselined[:]=0
all_noisy_psds=[]

for ip_n, _ in enumerate(left_ips):
    ips_pair.append([freqs[int(left_ips[ip_n])], freqs[int(right_ips[ip_n]+1)]]) 
    #+1 here because Iwilluse these values as range,and range in pythonis usually "up to the value but not including", this should fix it to the right range
    ips_l.append(freqs[int(left_ips[ip_n])])
    ips_r.append(freqs[int(right_ips[ip_n]+1)])
    avg_psd_cut[int(left_ips[ip_n]):int(right_ips[ip_n])+1]=None
    avg_psd_only_peaks[int(left_ips[ip_n]):int(right_ips[ip_n])+1]=avg_psd[int(left_ips[ip_n]):int(right_ips[ip_n])+1].copy()
    print([width_heights[ip_n]]*len(avg_psd_only_peaks[int(left_ips[ip_n]):int(right_ips[ip_n])+1]))
    avg_psd_only_peaks_baselined[int(left_ips[ip_n]):int(right_ips[ip_n])+1]=avg_psd_only_peaks[int(left_ips[ip_n]):int(right_ips[ip_n])+1]-[width_heights[ip_n]]*len(avg_psd_only_peaks[int(left_ips[ip_n]):int(right_ips[ip_n])+1])


avg_psd_only_peaks_baselined_new=np.array([avg_psd_only_peaks_baselined]) 

all_bandpower_per_ch_list=[]
all_rel_bandpower_per_ch_list=[]
for ip_n, _ in enumerate(left_ips):

    print('HERE!')
    print(avg_psd_only_peaks_baselined_new)
    bandpower_per_ch_list, power_by_Nfreq_per_ch_list, rel_bandpower_per_ch_list = Power_of_band(freqs=freqs, f_low = freqs[int(left_ips[ip_n])], f_high= freqs[int(right_ips[ip_n]+1)], psds=avg_psd_only_peaks_baselined_new)

    all_bandpower_per_ch_list.append(bandpower_per_ch_list)
    all_rel_bandpower_per_ch_list.append(rel_bandpower_per_ch_list)


print('Freq band for each peak:', ips_pair)
print('BP', all_bandpower_per_ch_list)
print('relative BP', all_rel_bandpower_per_ch_list)

plt.plot(freqs,avg_psd)
plt.plot(freqs[peaks], avg_psd[peaks], 'x')
plt.hlines(y=width_heights, xmin=ips_l, xmax=ips_r, color="C3")
plt.show()

plt.plot(freqs,avg_psd_cut)
plt.plot(freqs[peaks], avg_psd_cut[peaks], "x")
plt.hlines(y=width_heights, xmin=ips_l, xmax=ips_r, color="C3")
plt.show()

plt.plot(freqs,avg_psd_only_peaks)
plt.plot(freqs[peaks], avg_psd_only_peaks[peaks], "x")
plt.hlines(y=width_heights, xmin=ips_l, xmax=ips_r, color="C3")
plt.show()

plt.plot(freqs,avg_psd_only_peaks_baselined)
plt.plot(freqs[peaks], avg_psd_only_peaks_baselined[peaks], "x")
plt.show()

#NOW need power of freq. for each band in baselined




In [ ]:
from PSD_meg_qc import Power_of_band

bandpower_per_ch_list, power_by_Nfreq_per_ch_list, rel_bandpower_per_ch_list = Power_of_band(freqs=freqs, f_low = 57.5, f_high= 62.5, psds=avg_psd_new)

In [ ]:
#fit polynomial models up to degree 5
model1 = np.poly1d(np.polyfit(freqs, avg_psd, 1))
model2 = np.poly1d(np.polyfit(freqs, avg_psd, 2))
model3 = np.poly1d(np.polyfit(freqs, avg_psd, 3))
model4 = np.poly1d(np.polyfit(freqs, avg_psd, 4))
model5 = np.poly1d(np.polyfit(freqs, avg_psd, 5))

fig.add_trace(go.Scatter(x=freqs, y=model1(freqs), name='model1')) 
fig.add_trace(go.Scatter(x=freqs, y=model2(freqs), name='model2')) 
fig.add_trace(go.Scatter(x=freqs, y=model3(freqs), name='model3')) 
fig.add_trace(go.Scatter(x=freqs, y=model4(freqs), name='model4')) 
fig.add_trace(go.Scatter(x=freqs, y=model5(freqs), name='model5')) 




fig.show()

In [ ]:
def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    # if x.ndim != 1:
    #     raise ValueError, "smooth only accepts 1 dimension arrays."

    # if x.size < window_len:
    #     raise ValueError, "Input vector needs to be bigger than window size."


    # if window_len<3:
    #     return x


    # if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
    #     raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"


    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y



In [ ]:
avg_psd_smooth=smooth(avg_psd,window_len=11,window='hanning')
fig.add_trace(go.Scatter(x=freqs, y=avg_psd_smooth, name='smooth')) 
fig.show()


In [ ]:
import logging
import pandas as pd

# logging.basicConfig(datefmt='%H:%M:%S',
#                     stream=sys.stdout, level=logging.DEBUG,
#                     format='%(asctime)s %(message)s')

# Distance away from the FBEWMA that data should be removed.
DELTA = 0.1

# clip data above this value:
high_clip = 2.1

# clip data below this value:
LOW_CLIP = -2.1

# random values above this trigger a spike:
RAND_HIGH = 0.98

# random values below this trigger a negative spike:
RAND_LOW = 0.02

# How many samples to run the FBEWMA over.
SPAN = 10

# spike amplitude
SPIKE = 2


def clip_data(unclipped, high_clip, low_clip):
    ''' Clip unclipped between high_clip and low_clip. 
    unclipped contains a single column of unclipped data.'''
    
    # convert to np.array to access the np.where method
    np_unclipped = np.array(unclipped)
    # clip data above HIGH_CLIP or below LOW_CLIP
    cond_high_clip = (np_unclipped > high_clip) | (np_unclipped < low_clip)
    np_clipped = np.where(cond_high_clip, np.nan, np_unclipped)
    return np_clipped.tolist()


def create_sample_data():
    ''' Create sine wave, amplitude +/-2 with random spikes. '''
    x = np.linspace(0, 2*np.pi, 1000)
    y = 2 * np.sin(x)
    df = pd.DataFrame(list(zip(x,y)), columns=['x', 'y'])
    df['rand'] = np.random.random_sample(len(x),)
    # create random positive and negative spikes
    cond_spike_high = (df['rand'] > RAND_HIGH)
    df['spike_high'] = np.where(cond_spike_high, SPIKE, 0)
    cond_spike_low = (df['rand'] < RAND_LOW)
    df['spike_low'] = np.where(cond_spike_low, -SPIKE, 0)
    df['y_spikey'] = df['y'] + df['spike_high'] + df['spike_low']
    return df


def ewma_fb(df_column, span):
    ''' Apply forwards, backwards exponential weighted moving average (EWMA) to df_column. '''
    # Forwards EWMA.
    fwd = pd.Series.ewm(df_column, span=span).mean()
    # Backwards EWMA.
    bwd = pd.Series.ewm(df_column[::-1],span=10).mean()
    # Add and take the mean of the forwards and backwards EWMA.
    stacked_ewma = np.vstack(( fwd, bwd[::-1] ))
    fb_ewma = np.mean(stacked_ewma, axis=0)
    return fb_ewma
    
    
def remove_outliers(spikey, fbewma, delta):
    ''' Remove data from df_spikey that is > delta from fbewma. '''
    np_spikey = np.array(spikey)
    np_fbewma = np.array(fbewma)
    cond_delta = (np.abs(np_spikey-np_fbewma) > delta)
    np_remove_outliers = np.where(cond_delta, np.nan, np_spikey)
    return np_remove_outliers

    
def main(avg_psd, freqs):
    #df = create_sample_data()
    df = pd.DataFrame(list(zip(freqs,avg_psd)), columns=['x', 'y'])
    df['y_spikey']=avg_psd

    df['y_clipped'] = clip_data(df['y_spikey'].tolist(), HIGH_CLIP, LOW_CLIP)
    df['y_ewma_fb'] = ewma_fb(df['y_clipped'], SPAN)
    df['y_remove_outliers'] = remove_outliers(df['y_clipped'].tolist(), df['y_ewma_fb'].tolist(), DELTA)
    df['y_interpolated'] = df['y_remove_outliers'].interpolate()
    
    ax = df.plot(x='x', y='y_spikey', color='blue', alpha=0.5)
    ax2 = df.plot(x='x', y='y_interpolated', color='black', ax=ax)

    return df
    
df=main(avg_psd, freqs)

df

In [ ]:
import logging
import numpy as np
import pandas as pd

logging.basicConfig(datefmt='%H:%M:%S',
                    stream=sys.stdout, level=logging.DEBUG,
                    format='%(asctime)s %(message)s')

# Distance away from the FBEWMA that data should be removed.
DELTA = 0.1

# clip data above this value:
HIGH_CLIP = 2.1

# clip data below this value:
LOW_CLIP = -2.1

# random values above this trigger a spike:
RAND_HIGH = 0.98

# random values below this trigger a negative spike:
RAND_LOW = 0.02

# How many samples to run the FBEWMA over.
SPAN = 10

# spike amplitude
SPIKE = 2


def clip_data(unclipped, high_clip, low_clip):
    ''' Clip unclipped between high_clip and low_clip. 
    unclipped contains a single column of unclipped data.'''
    
    # convert to np.array to access the np.where method
    np_unclipped = np.array(unclipped)
    # clip data above HIGH_CLIP or below LOW_CLIP
    cond_high_clip = (np_unclipped > HIGH_CLIP) | (np_unclipped < LOW_CLIP)
    np_clipped = np.where(cond_high_clip, np.nan, np_unclipped)
    return np_clipped.tolist()


def create_sample_data():
    ''' Create sine wave, amplitude +/-2 with random spikes. '''
    x = np.linspace(0, 2*np.pi, 1000)
    y = 2 * np.sin(x)
    df = pd.DataFrame(list(zip(x,y)), columns=['x', 'y'])
    df['rand'] = np.random.random_sample(len(x),)
    # create random positive and negative spikes
    cond_spike_high = (df['rand'] > RAND_HIGH)
    df['spike_high'] = np.where(cond_spike_high, SPIKE, 0)
    cond_spike_low = (df['rand'] < RAND_LOW)
    df['spike_low'] = np.where(cond_spike_low, -SPIKE, 0)
    df['y_spikey'] = df['y'] + df['spike_high'] + df['spike_low']

    return df


def ewma_fb(df_column, span):
    ''' Apply forwards, backwards exponential weighted moving average (EWMA) to df_column. '''
    # Forwards EWMA.
    fwd = pd.Series.ewm(df_column, span=span).mean()
    # Backwards EWMA.
    bwd = pd.Series.ewm(df_column[::-1],span=10).mean()
    # Add and take the mean of the forwards and backwards EWMA.
    stacked_ewma = np.vstack(( fwd, bwd[::-1] ))
    fb_ewma = np.mean(stacked_ewma, axis=0)
    return fb_ewma
    
    
def remove_outliers(spikey, fbewma, delta):
    ''' Remove data from df_spikey that is > delta from fbewma. '''
    np_spikey = np.array(spikey)
    np_fbewma = np.array(fbewma)
    cond_delta = (np.abs(np_spikey-np_fbewma) > delta)
    np_remove_outliers = np.where(cond_delta, np.nan, np_spikey)
    return np_remove_outliers

    
def main():
    df = create_sample_data()

    df['y_clipped'] = clip_data(df['y_spikey'].tolist(), HIGH_CLIP, LOW_CLIP)
    df['y_ewma_fb'] = ewma_fb(df['y_clipped'], SPAN)
    df['y_remove_outliers'] = remove_outliers(df['y_clipped'].tolist(), df['y_ewma_fb'].tolist(), DELTA)
    df['y_interpolated'] = df['y_remove_outliers'].interpolate()
    
    ax = df.plot(x='x', y='y_spikey', color='blue', alpha=0.5)
    ax2 = df.plot(x='x', y='y_interpolated', color='black', ax=ax)

    return df
    
df= main()

df

In [ ]:
from PSD_meg_qc import Power_of_band
avg_psd_new=np.array([avg_psd])
bandpower_per_ch_list, power_by_Nfreq_per_ch_list, rel_bandpower_per_ch_list = Power_of_band(freqs=freqs, f_low = 57.5, f_high= 62.5, psds=avg_psd_new)

bandpower_per_ch_list

In [ ]:
#plot EOG channel

raw.copy().pick_types(meg=False, stim=False,eog=True).plot()

In [ ]:
from ECG_meg_qc import find_affected_channels

mag_ch_names = raw.copy().pick_types(meg='mag').ch_names if 'mag' in raw else None
grad_ch_names = raw.copy().pick_types(meg='grad').ch_names if 'grad' in raw else None
channels = {'mag': mag_ch_names, 'grad': grad_ch_names}
sfreq=raw.info['sfreq']
m_or_g='mag'
tmin=-0.2
tmax=0.2
#ecg_epochs = mne.preprocessing.create_ecg_epochs(raw, picks=channels[m_or_g], tmin=tmin, tmax=tmax)

#ch_name='EOG 061'
ch_name='EOG002'
eog_epochs = mne.preprocessing.create_eog_epochs(raw, picks=channels[m_or_g], ch_name=ch_name, tmin=tmin, tmax=tmax)


ecg_affected_channels, fig_affected, fig_not_affected, fig_avg=find_affected_channels(eog_epochs, channels, m_or_g, norm_lvl=1, ecg_or_eog='EOG', thresh_lvl_peakfinder=5, sfreq=sfreq, tmin=tmin, tmax=tmax, plotflag=True,  use_abs_of_all_data='flip')


In [ ]:
ch_name='EOG003'
eog_epochs = mne.preprocessing.create_eog_epochs(raw, picks=channels[m_or_g], ch_name=ch_name, tmin=tmin, tmax=tmax)


ecg_affected_channels, fig_affected, fig_not_affected, fig_avg=find_affected_channels(eog_epochs, channels, m_or_g, norm_lvl=1, ecg_or_eog='EOG', thresh_lvl_peakfinder=5, sfreq=sfreq, tmin=tmin, tmax=tmax, plotflag=True,  use_abs_of_all_data='flip')


In [ ]:

eog_epochs = mne.preprocessing.create_eog_epochs(raw, picks=channels[m_or_g], tmin=tmin, tmax=tmax)


ecg_affected_channels, fig_affected, fig_not_affected, fig_avg=find_affected_channels(eog_epochs, channels, m_or_g, norm_lvl=1, ecg_or_eog='EOG', thresh_lvl_peakfinder=5, sfreq=sfreq, tmin=tmin, tmax=tmax, plotflag=True,  use_abs_of_all_data='flip')


In [ ]:
ecg_affected_channels, fig_affected, fig_not_affected, fig_avg=find_affected_channels(ecg_epochs, channels, m_or_g, norm_lvl=1, ecg_or_eog='ecg', thresh_lvl_peakfinder=5, sfreq=sfreq, tmin=tmin, tmax=tmax, plotflag=True, use_abs_of_all_data='False')


In [ ]:
ecg_affected_channels, fig_affected, fig_not_affected, fig_avg=find_affected_channels(ecg_epochs, channels, m_or_g, norm_lvl=1, ecg_or_eog='ecg', thresh_lvl_peakfinder=5, sfreq=sfreq, tmin=tmin, tmax=tmax, plotflag=True, use_abs_of_all_data='flip')


In [ ]:
data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003682/sub-001/ses-01/meg/sub-001_ses-01_task-AversiveLearningReplay_run-01_meg.fif'
raw = mne.io.read_raw_fif(data_file)
raw.info['chs']

In [ ]:


import numpy as np
import mne
y=np.array([1,4,5,0,0,0])
peaks_l, peaks_m = mne.preprocessing.peak_finder(y)

print(peaks_l)
print(type(peaks_l[0]))
print(len(peaks_l))

print(np.array([87]))
print(type(np.array([87])[0]))
#print(len(np.array(87)))


print(np.ndarray(87))
print(len(np.ndarray(87)))

In [ ]:
# Other useful ancp stuff:

config = configparser.ConfigParser()
config.read('settings.ini')

default_direct = config['DEFAULT']['data_directory']
dataset_path = ancpbids.utils.fetch_dataset(default_direct)

from ancpbids import BIDSLayout
layout = BIDSLayout(dataset_path)

list_of_fifs = layout.get(suffix='meg', extension='.fif', return_type='filename')

list_of_subs = layout.get_subjects()


list_of_entities = layout.get_entities()
print(list_of_entities)

In [ ]:
#TRY SEPARATE FUNCS HERE

config = configparser.ConfigParser()
config.read('settings.ini')
data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003483/sub-009/ses-1/meg/sub-009_ses-1_task-deduction_run-1_meg.fif'
all_qc_params = get_all_config_params('settings.ini')
dict_of_dfs_epoch, dict_epochs_mg, channels, raw_filtered, raw_filtered_resampled, raw_cropped, raw, active_shielding_used = initial_processing(default_settings=all_qc_params['default'], filtering_settings=all_qc_params['Filtering'], epoching_params=all_qc_params['Epoching'], data_file=data_file)

m_or_g_chosen = ['mag']
sid='009'


In [ ]:
start_time = time.time()


#dfs_ptp_amlitude_annot, bad_channels, amplit_annot_with_ch_names = PP_auto_meg_qc(sid, config, channels, raw, m_or_g_chosen)

#derivs, big_rmse_with_value_all_data, small_rmse_with_value_all_data = RMSE_meg_qc(all_qc_params['RMSE'], channels, dict_epochs_mg, dict_dict_of_dfs_epoch, raw, m_or_g_chosen)

derivs = PP_manual_meg_qc(all_qc_params['PTP_manual'], channels, dict_epochs_mg, dict_of_dfs_epoch, raw, m_or_g_chosen)

#out_with_name_and_format = PSD_meg_qc(sid, config, channels, raw, m_or_g_chosen)

derivs = ECG_meg_qc(config, raw, m_or_g_chosen)

#out_with_name_and_format = EOG_meg_qc(config, raw, m_or_g_chosen)

#out_with_name_and_format, bad_channels = PP_auto_meg_qc(sid, config, channels, raw, m_or_g_chosen)

print("--- Execution %s seconds ---" % (time.time() - start_time))


for der in derivs:
    if der.content_type == 'plotly':
        der.content.show()


In [ ]:
# out_with_name_and_format[0].convert_fig_to_html()
from universal_plots import QC_derivative
dr = QC_derivative('001', 'mean_EC_epoch', None, '')
# dr_html = dr.convert_fig_to_html()
# l=["a"]

# if dr_html is not None:

#     l += dr_html
    
# print(l)

sec = dr.get_section()
print(sec)


In [ ]:
print(figs)
all_fig_derivs = figs
figures_report = {}
for x in range(0, len(all_fig_derivs)):
    if all_fig_derivs[x][3]=='plotly':
        figures_report["f{0}".format(x)] = plotly.io.to_html(all_fig_derivs[x][0])
    elif all_fig_derivs[x][3]=='matplotlib':
        figures_report["f{0}".format(x)] = mpld3.fig_to_html(all_fig_derivs[x][0]);

print(figures_report)

In [ ]:

import mpld3

html_fig=mpld3.fig_to_html(figs[0][0])
print(html_fig)



# file = open('matpl_fig.html', "w")
# file.write(html_fig)
# file.close()

In [ ]:
html_string = '''
<!doctype html>
<html>
    <head>
        <meta charset="UTF-8">
        <title>MEG QC: Frequency spectrum Report</title>
        <style>body{ margin:0 100;}</style>
    </head>
    
    <body style="font-family: Arial">
        <center>
        <h1>MEG data quality analysis report</h1>
        <br></br>
        <!-- *** Section 1 *** --->
        <h2>Frequency spectrum per channel</h2>
        ''' + html_fig + '''
        <p>graph description...</p>
        </center>
    
    </body>
</html>'''

with open('report_trial.html', 'w', encoding = 'utf8') as f:
    f.write(html_string)

In [ ]:
#config = configparser.ConfigParser()
#config.read('settings.ini')

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds000117/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_meg.fif'
# file does not start with a file id tag

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003392/sub-01/meg/sub-01_task-localizer_meg.fif'
# SSS frilter. need to allow maxshiled.

#data_file ='/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003694/sub-01/meg/sub-01_task-MEM_run-01_meg.fif'
#raw = mne.io.read_raw_fif(data_file, on_split_missing='ignore')


#dict_of_dfs_epoch, dict_epochs_mg, channels, raw_bandpass, raw_filtered_resampled, raw_cropped, raw = initial_processing(config, data_file)

#data_file ='/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003922/sub-Mp150285/ses-01/meg/sub-Mp150285_ses-01_acq-crosstalk_meg.fif'
#Could not find measurement data: how many in this set, which subjects?


data_file ='/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds004229/sub-102/meg/sub-102_task-amnoise_meg.fif'
#SSS filter
raw = mne.io.read_raw_fif(data_file, allow_maxshield=True)


#data_dir ='/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/not fitting no fif/ds000246/sub-0001/meg/sub-0001_task-AEF_run-01_meg.ds'
#raw = mne.io.read_raw_ctf(data_dir)


In [ ]:
import configparser
from ancpbids import BIDSLayout

config = configparser.ConfigParser()
config.read('settings.ini')

default_section = config['DEFAULT']

m_or_g_chosen = default_section['do_for'] 
m_or_g_chosen = m_or_g_chosen.replace(" ", "")
m_or_g_chosen = m_or_g_chosen.split(",")
#m_or_g_chosen = select_m_or_g(default_section)

dataset_path = default_section['data_directory']

layout = BIDSLayout(dataset_path)
schema = layout.schema

#create derivative folder first!
derivative = layout.dataset.create_derivative(name="Meg_QC")
derivative.dataset_description.GeneratedBy.Name = "MEG QC Pipeline"

list_of_subs = layout.get_subjects()
#print(list_of_subs)
for sub in list_of_subs:
    list_of_fifs_per_sub = layout.get(suffix='meg', extension='.fif', sub=sub)
    for_entities={}
    for file in list_of_fifs_per_sub:
        k=file['entities'][1]['key']
        v=file['entities'][1]['value']
        for_entities[k]=v
    print(for_entities)

#print(list_of_fifs_per_sub)
# for sid in [list_of_subs[0]]: #RUN OVER JUST 1 SUBJ
# #for sid in list_of_subs: 

#     subject_folder = derivative.create_folder(type_=schema.Subject, name='sub-'+sid)

#     list_of_fifs = layout.get(suffix='meg', extension='.fif', return_type='filename', subj=sid)
#     #Devide here fifs by task, ses , run

#     for data_file in [list_of_fifs[0]]: #RUN OVER JUST 1 FIF because is not divided by tasks yet..
#         print(type(data_file))